In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
root = "/content/drive/MyDrive/COMP721_Project"

In [3]:
data = os.path.join(root, "Data")

#preprocessing

In [22]:
player_rs_career = pd.read_csv(os.path.join(data, "player_regular_season_career.csv"))
player_rs_career.head()

,ilkid,firstname,lastname,leag,gp,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
0,ABDELAL01,Alaa,Abdelnaby,N,256,3200,1465,283,563,846,85,71,69,247,484,1236,620,321,225,3,0
1,ABDULKA01,Kareem,Abdul-jabbar,N,1560,57446,38387,2975,9394,17440,5660,1160,3189,2527,4657,28307,15837,9304,6712,18,1
2,ABDULMA01,Mahmo,Abdul-rauf,N,586,15633,8553,219,868,1087,2079,487,46,963,1107,7943,3514,1161,1051,1339,474
3,ABDULTA01,Tariq,Abdul-wahad,N,236,4808,1830,275,501,776,266,184,82,309,485,1726,720,529,372,76,18
4,ABDURSH01,Shareef,Abdur-rahim,N,672,24862,13338,1498,3976,5474,1847,718,556,1911,1845,10215,4789,4427,3614,477,146


In [23]:
player_keys = player_rs_career.loc[:,:"leag"]
player_vecs = player_rs_career.loc[:,"gp":]
player_vecs.head()

,gp,minutes,pts,oreb,dreb,reb,asts,stl,blk,turnover,pf,fga,fgm,fta,ftm,tpa,tpm
0,256,3200,1465,283,563,846,85,71,69,247,484,1236,620,321,225,3,0
1,1560,57446,38387,2975,9394,17440,5660,1160,3189,2527,4657,28307,15837,9304,6712,18,1
2,586,15633,8553,219,868,1087,2079,487,46,963,1107,7943,3514,1161,1051,1339,474
3,236,4808,1830,275,501,776,266,184,82,309,485,1726,720,529,372,76,18
4,672,24862,13338,1498,3976,5474,1847,718,556,1911,1845,10215,4789,4427,3614,477,146


In [ ]:
from sklearn.decomposition import FastICA

#Models

##One-Class SVM

In [ ]:
from sklearn import svm

In [7]:
model = svm.OneClassSVM()

## Isolation Forest

In [24]:
from sklearn.ensemble import IsolationForest

In [26]:
model = IsolationForest()

## Covariance

In [30]:
from sklearn.covariance import EllipticEnvelope

In [31]:
model = EllipticEnvelope()

## 


#Testing

In [32]:
res = model.fit_predict(player_vecs)
res

/usr/local/lib/python3.7/dist-packages/sklearn/covariance/_robust_covariance.py:189: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (65.158623952016796 > 64.739003668089751). You may want to try with a higher value of support_fraction (current value: 0.505).
  RuntimeWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/covariance/_robust_covariance.py:189: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (69.335237950371322 > 66.778687739813762). You may want to try with a higher value of support_fraction (current value: 0.505).
  RuntimeWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/covariance/_robust_covariance.py:189: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (84.849632590089470 > 83.445243271202173). You may want to try with a higher value of support_fraction (current value: 0.505).
  RuntimeWarning,
/usr/local/lib/python3

array([ 1, -1,  1, ...,  1,  1,  1])

In [33]:
players = player_keys.copy()
players["outlier"] = res
players.head(10)

,ilkid,firstname,lastname,leag,outlier
0,ABDELAL01,Alaa,Abdelnaby,N,1
1,ABDULKA01,Kareem,Abdul-jabbar,N,-1
2,ABDULMA01,Mahmo,Abdul-rauf,N,1
3,ABDULTA01,Tariq,Abdul-wahad,N,1
4,ABDURSH01,Shareef,Abdur-rahim,N,-1
5,ABERNTO01,Tom,Abernethy,N,1
6,ABLEFO01,Forest,Able,N,1
7,ABRAMJO01,John,Abramovic,N,1
8,ACKERDO01,Donald,Ackerman,N,1
9,ACRESMA01,Mark,Acres,N,1
